Тестовая реализация метода обратного распространения ошибки 
для двухслойной полностью связанной нейронной сети

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

Что такое MNIST и как его загружать

In [ ]:
# Загрузка данных MNIST
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:

# Преобразование входных изображений в одномерный массив и нормализация пикселей
x_train = x_train.reshape(-1, 784) / 255.0 
x_test = x_test.reshape(-1, 784) / 255.0

Определяем архитектуру модели

С помощью инструментов keras.Sequential определяем последовательную модель нейронной сети, который добавляет два полностью связанных слоя, каждый с определенным числом нейронов.

Первый слой использует функцию активации sigmoid и принимает входные данные в форме вектора размерности 784 (28 на 28).

Второй слой обычно используется в задачах классификации и использует функцию активации softmax, чтобы вычислить вероятности принадлежности объектов к тому или иному классу. 

In [ ]:

# Определение архитектуры модели
model = keras.Sequential([
    layers.Dense(128, activation='sigmoid', input_shape=(784,)),
    layers.Dense(10, activation='softmax'),
])

In [ ]:

# Компиляция модели
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели на тренировочных данных
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Оценка модели на тестовых данных
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

# Расчет градиента потерь по отношению к весам модели в выходных слоях
with tf.GradientTape() as tape:
    logits = model(x_train, training=False)
    loss_value = keras.backend.sparse_categorical_crossentropy(y_train, logits)

grads = tape.gradient(loss_value, model.trainable_weights)

# Обновление весов модели на основе градиентов
model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

# Повторное обучение модели с обновленными весами
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)